## Game prediction NBA 2023
#### Amber Agusti, ...

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
ratings = pd.read_table('team_rtgs.txt', ',')
ratings.head()

#looking primarily ar offensive and defensive adjusted ratings
#adjusted for opposition's offensive/ defensive rating


,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,1,Boston Celtics,E,A,42,17,0.712,6.14,118.88,112.47,6.40,5.78,119.19,113.14,6.06
1,2,Cleveland Cavaliers,E,C,38,23,0.623,5.62,116.78,110.92,5.86,5.60,116.81,110.97,5.84
2,3,Denver Nuggets,W,NW,41,18,0.695,4.36,119.08,114.73,4.36,3.98,118.78,114.82,3.96
3,4,Memphis Grizzlies,W,SW,35,22,0.614,3.91,114.83,110.95,3.88,3.80,114.41,110.65,3.76
4,5,Philadelphia 76ers,E,A,38,19,0.667,3.89,117.40,113.50,3.90,3.52,117.13,113.59,3.54


In [3]:
#tables contains monthly NBA game results
oct_games = pd.read_table('oct_games.txt', ',') 
nov_games = pd.read_table('nov_games.txt', ',')
dec_games = pd.read_table('dec_games.txt', ',')
jan_games = pd.read_table('jan_games.txt', ',')
feb_games = pd.read_table('feb_games.txt', ',') # only added games before 2/24 (completed games)


#concatenating three dfs
games = oct_games
games = pd.concat([games, nov_games], join = 'inner')
games = pd.concat([games, dec_games], join = 'inner')
games = pd.concat([games, jan_games], join = 'inner')
games = pd.concat([games, feb_games], join = 'inner')

pd.set_option('display.max_rows', None)
display(games)

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes
0,Tue Oct 18 2022,7:30p,Philadelphia 76ers,117,Boston Celtics,126,Box Score,NaN,19156.0,TD Garden,NaN
1,Tue Oct 18 2022,10:00p,Los Angeles Lakers,109,Golden State Warriors,123,Box Score,NaN,18064.0,Chase Center,NaN
2,Wed Oct 19 2022,7:00p,Orlando Magic,109,Detroit Pistons,113,Box Score,NaN,20190.0,Little Caesars Arena,NaN
3,Wed Oct 19 2022,7:00p,Washington Wizards,114,Indiana Pacers,107,Box Score,NaN,15027.0,Gainbridge Fieldhouse,NaN
4,Wed Oct 19 2022,7:30p,Houston Rockets,107,Atlanta Hawks,117,Box Score,NaN,17878.0,State Farm Arena,NaN
5,Wed Oct 19 2022,7:30p,New Orleans Pelicans,130,Brooklyn Nets,108,Box Score,NaN,18003.0,Barclays Center,NaN
6,Wed Oct 19 2022,7:30p,New York Knicks,112,Memphis Grizzlies,115,Box Score,OT,18202.0,FedEx Forum,NaN
7,Wed Oct 19 2022,7:30p,Chicago Bulls,116,Miami Heat,108,Box Score,NaN,19600.0,FTX Arena,NaN
8,Wed Oct 19 2022,7:30p,Cleveland Cavaliers,105,Toronto Raptors,108,Box Score,NaN,19800.0,Scotiabank Arena,NaN
9,Wed Oct 19 2022,8:00p,Oklahoma City Thunder,108,Minnesota Timberwolves,115,Box Score,NaN,17136.0,Target Center,NaN


In [4]:
#adding new_col that states 1 for win 0 for loss for the home team 
#adding new_col2 that shows the winning team
new_col = []
new_col2 = []

for i in range(games.shape[0]):
    if list(games['PTS'])[i] < list(games['PTS.1'])[i]:
        new_col.append(1)
        new_col2.append(list(games['Home/Neutral'])[i])
    else: 
        new_col.append(0)
        new_col2.append(list(games['Visitor/Neutral'])[i])
        
games['win/loss'] = new_col
games['winning team'] = new_col2
games.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes,win/loss,winning team
0,Tue Oct 18 2022,7:30p,Philadelphia 76ers,117,Boston Celtics,126,Box Score,NaN,19156.0,TD Garden,NaN,1,Boston Celtics
1,Tue Oct 18 2022,10:00p,Los Angeles Lakers,109,Golden State Warriors,123,Box Score,NaN,18064.0,Chase Center,NaN,1,Golden State Warriors
2,Wed Oct 19 2022,7:00p,Orlando Magic,109,Detroit Pistons,113,Box Score,NaN,20190.0,Little Caesars Arena,NaN,1,Detroit Pistons
3,Wed Oct 19 2022,7:00p,Washington Wizards,114,Indiana Pacers,107,Box Score,NaN,15027.0,Gainbridge Fieldhouse,NaN,0,Washington Wizards
4,Wed Oct 19 2022,7:30p,Houston Rockets,107,Atlanta Hawks,117,Box Score,NaN,17878.0,State Farm Arena,NaN,1,Atlanta Hawks


In [5]:
team_ratings = dict()
for i in range(ratings.shape[0]):
    team_ratings[ratings['Team'][i]] = (ratings['ORtg/A'][i], ratings['DRtg/A'][i])
    
#team_ratings is a dict where key is the team
#val is a tuple consisting of offensive and defensive rating respectfully

##### I wonder how we would/ could/ should use nonstatic ratings?

In [6]:
#creating X

home_ORtgA = [] #contains offensive adjusted raings for home/ away teams for games
away_ORtgA = []
for i in range(games.shape[0]):
    away_ORtgA.append(team_ratings[list(games['Visitor/Neutral'])[i]][0])
    home_ORtgA.append(team_ratings[list(games['Home/Neutral'])[i]][0])

#creating X where X contains offensive adj ratings for both teams
X = np.array([home_ORtgA, away_ORtgA]).T

y = list(games['win/loss'])


In [7]:
#trying log regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.6145251396648045

In [8]:
#adding defensive adj ratings to X

home_DRtgA = [] #contains defensive adjusted raings for home/ away teams for games
away_DRtgA = []
for i in range(games.shape[0]):
    away_DRtgA.append(team_ratings[list(games['Visitor/Neutral'])[i]][1])
    home_DRtgA.append(team_ratings[list(games['Home/Neutral'])[i]][1])
    
#creating X where X contains offensive adj ratings for both teams
X2 = np.array([home_ORtgA, away_ORtgA, home_DRtgA, away_DRtgA]).T

In [9]:
#trying log regression with X2 (offense and defense adj ratings both teams)

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2)

model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.6703910614525139

In [17]:
#predicting whether home team (Lakers) will win (1) / lose (0)


X_new = np.array([[team_ratings['Los Angeles Lakers'][0], team_ratings['Washington Wizards'][0], team_ratings['Los Angeles Lakers'][1], team_ratings['Washington Wizards'][1]]])

model.predict(X_new)

array([1])